In [1]:
from __future__ import print_function 
import os, sys
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf

sys.path.append(os.path.join('..','..'))
from tfomics import layers, utils, init, learn
from tfomics import neuralnetwork as nn

# load data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

X_train = mnist.train.images
X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
y_train = mnist.train.labels

X_valid = mnist.validation.images
X_valid = np.reshape(X_valid, (X_valid.shape[0], 28, 28, 1))
y_valid = mnist.validation.labels

X_test = mnist.test.images
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))
y_test = mnist.test.labels

# get shapes
num_data, height, widht, dim = X_train.shape
input_shape=[None, height, widht, dim]
num_labels = y_train.shape[1]   # number of labels (output units)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# build network

In [3]:
# design a neural network model
input_vars = utils.placeholder(shape=input_shape, name='input')
target_vars = utils.placeholder(shape=(None,num_labels), name='output')
is_training = tf.placeholder(tf.bool, name='is_training')   # variable to specify training or testing
keep_prob = tf.placeholder(tf.float32, name='keep_prob')   # dropout probability
placeholders = [input_vars, target_vars, is_training, keep_prob] 

net = OrderedDict()
net['inputs'] = layers.InputLayer(input_vars)
net['conv1'] = layers.Conv2DLayer(net['inputs'], filter_size=(5,5), num_filters=32, padding='SAME')
net['conv1_norm'] = layers.BatchNormLayer(net['conv1'], is_training)
net['conv1_active'] = layers.ActivationLayer(net['conv1_norm'], function='relu')
net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_active'], pool_size=(2,2))

net['conv2'] = layers.Conv2DLayer(net['conv1_pool'], filter_size=(5,5), num_filters=64, 
                                  strides=(2,2), padding='VALID')
net['conv2_norm'] = layers.BatchNormLayer(net['conv2'], is_training)
net['conv2_active'] = layers.ActivationLayer(net['conv2_norm'], function='relu')
net['conv2_dropout'] = layers.DropoutLayer(net['conv2_active'], keep_prob=keep_prob)

net['dense1'] = layers.DenseLayer(net['conv2_dropout'], num_units=512, W=init.HeNormal(), b=None)
net['dense1_norm'] = layers.BatchNormLayer(net['dense1'], is_training)
net['dense1_active'] = layers.ActivationLayer(net['dense1_norm'], function='relu')
net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], keep_prob=keep_prob)

net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, 
                                  W=init.HeNormal(), b=init.Constant(0.05))
net['output'] = layers.ActivationLayer(net['dense2'], function='sigmoid')

optimization = {"objective": "categorical",  # (binary, categorical, squared_error)
                "optimizer": "adam",    
                "learning_rate": 0.001, # learning rate
                "clip_value": True,     # clip prediction values (True for classification)
                "l2": .00001            # l-2 weight decay
                }

In [4]:
# build neural network class
nnmodel = nn.NeuralNet(net, [input_vars])
nnmodel.inspect_layers()

# compile neural trainer
save_path = 'results'
if not os.path.isdir(save_path):
    os.mkdir(save_path)
filename = 'test'
filepath = os.path.join(save_path, filename)
nntrainer = nn.NeuralTrainer(nnmodel, target_vars, placeholders, optimization, 
                             save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: inputs
(?, 28, 28, 1)
layer2: conv1
(?, 28, 28, 32)
layer3: conv1_norm
(?, 28, 28, 32)
layer4: conv1_active
(?, 28, 28, 32)
layer5: conv1_pool
(?, 14, 14, 32)
layer6: conv2
(?, 5, 5, 64)
layer7: conv2_norm
(?, 5, 5, 64)
layer8: conv2_active
(?, 5, 5, 64)
layer9: conv2_dropout
(?, 5, 5, 64)
layer10: dense1
(?, 512)
layer11: dense1_norm
(?, 512)
layer12: dense1_active
(?, 512)
layer13: dense1_dropout
(?, 512)
layer14: dense2
(?, 10)
layer15: output
(?, 10)
----------------------------------------------------------------------------


In [5]:
# run session
sess = tf.Session()

# initialize variables
sess.run(tf.initialize_all_variables())

In [6]:
# organize dataset to be just like placeholders list
data = {'train':  [X_train, y_train, True, 0.7], 'valid': [X_valid, y_valid, False, 1]}
learn.train_minibatch(sess, nntrainer, data, batch_size=128, num_epochs=500, 
                      patience=10, verbose=1, shuffle=True)

Epoch 1 out of 500 


ResourceExhaustedError: OOM when allocating tensor with shape[128,28,28,32]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_input_0/_5, Variable/read)]]
	 [[Node: moments_1/sufficient_statistics/Shape/_13 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_906_moments_1/sufficient_statistics/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Conv2D', defined at:
  File "/home/peter/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/peter/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/peter/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <m

In [ ]:
# load best model
nnmodel.load_model_parameters(sess, filepath+'_best.ckpt')

test = [X_test, y_test, False, 1]
test_loss = nntrainer.test_model(sess, test, batch_size=512)

# Residual blocks

In [ ]:
# design a neural network model
input_vars = utils.placeholder(shape=input_shape, name='input')
target_vars = utils.placeholder(shape=(None,num_labels), name='output')
is_training = tf.placeholder(tf.bool, name='is_training')   # variable to specify training or testing
keep_prob = tf.placeholder(tf.float32, name='keep_prob')   # dropout probability
placeholders = [input_vars, target_vars, is_training, keep_prob] 

net = OrderedDict()
net['inputs'] = layers.InputLayer(input_vars)

# 1st convolution layer
net['conv1'] = layers.Conv2DLayer(net['inputs'], filter_size=(5,5), num_filters=32, padding='SAME')
net['conv1_norm'] = layers.BatchNormLayer(net['conv1'], is_training)
net['conv1_active'] = layers.ActivationLayer(net['conv1_norm'], function='relu')

# residual block 1
num_filters = 32
filter_size = (5,5)
last_layer = 'conv1_active'
name = 'conv1_2'
net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], filter_size=filter_size, num_filters=num_filters, padding='SAME')
net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'], is_training)
net[name+'_1resid_active'] = layers.ActivationLayer(net[name+'_1resid_norm'], function='relu')
net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], keep_prob=keep_prob)
net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], filter_size=filter_size, num_filters=num_filters, padding='SAME')
net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'], is_training)
net[name+'_residual'] = layers.ElementwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
net[name+'_resid'] = layers.ActivationLayer(net[name+'_residual'], function='relu')

net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_2_resid'], pool_size=(2,2))
net['conv1_dropout'] = layers.DropoutLayer(net['conv1_pool'], keep_prob=keep_prob)

# 2nd convolution layer
net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], filter_size=(5,5), num_filters=64, 
                                  strides=(1,1), padding='VALID')
net['conv2_norm'] = layers.BatchNormLayer(net['conv2'], is_training)
net['conv2_active'] = layers.ActivationLayer(net['conv2_norm'], function='relu')

# residual block 2
num_filters = 64
filter_size = (5,5)
last_layer = 'conv2_active'
name = 'conv2_2'
net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], filter_size=filter_size, num_filters=num_filters, padding='SAME')
net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'], is_training)
net[name+'_1resid_active'] = layers.ActivationLayer(net[name+'_1resid_norm'], function='relu')
net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], keep_prob=keep_prob)
net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], filter_size=filter_size, num_filters=num_filters, padding='SAME')
net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'], is_training)
net[name+'_residual'] = layers.ElementwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
net[name+'_resid'] = layers.ActivationLayer(net[name+'_residual'], function='relu')

net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(2,2))
net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], keep_prob=keep_prob)

# dense layer 1
net['dense1'] = layers.DenseLayer(net['conv2_dropout'], num_units=128, 
                                  W=init.HeNormal(), b=init.Constant(0.05))
net['dense1_norm'] = layers.BatchNormLayer(net['dense1'], is_training)
net['dense1_active'] = layers.ActivationLayer(net['dense1_norm'], function='relu')
net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], keep_prob=keep_prob)

# dense layer 2
net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, 
                                  W=init.HeNormal(), b=init.Constant(0.05))
net['output'] = layers.ActivationLayer(net['dense2'], function='sigmoid')


optimization = {"objective": "categorical",  # (binary, categorical, squared_error)
                "optimizer": "adam",    
                "learning_rate": 0.001, # learning rate
                "clip_value": True,     # clip prediction values (True for classification)
                "l2": .00001            # l-2 weight decay
                }


In [ ]:
# build neural network class
nnmodel = nn.NeuralNet(net, [input_vars])
nnmodel.inspect_layers()

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, target_vars, placeholders, optimization, 
                             save='best', filepath='.')

In [ ]:
# run session
sess = tf.Session()

# initialize variables
sess.run(tf.global_variables_initializer())

# organize dataset to be just like placeholders list
data = {'train':  [X_train, y_train, True, 0.3], 'valid': [X_valid, y_valid, False, 1]}
learn.train_minibatch(sess, nntrainer, data, batch_size=128, num_epochs=500, 
                      patience=10, verbose=1, shuffle=True)

In [ ]:
test = [X_test, y_test, False, 1]
test_loss = nntrainer.test_model(sess, test, batch_size=512)